<a href="https://colab.research.google.com/github/lcqsigi/big-data2/blob/main/063_Zillow_For_Sale_Properties_Python3.091624.bis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Steps
### Get Keys
1) Get Free [Rapid API Key by creating an account](https://rapidapi.com/marketplace)

Store each key in a central and private location.

### Subscribe to APIs

1) Subscribe to [Zillow on Rapid API](https://rapidapi.com/apimaker/api/zillow-com1/)

## <font color="blue">Install Packages</font>

## <font color="blue">Imports</font>

In [1]:
from google.colab import drive, files # specific to Google Colab
import pandas as pd
import requests
import json
import time

# show all columns
pd.set_option('display.max_columns', None)

## <font color="blue">Functions</font>

## <font color="blue">Local & Constants</font>

In [2]:
############
# OPTIONAL #
############

# mount drive
drive.mount('/content/drive', force_remount=False)

# data location
file_dir = '/content/drive/My Drive/' # optional

############
# REQURIED #
############
city = 'hayward'
state = 'ca'
search_str = city + ', ' + state
print('Search string:', search_str)

Mounted at /content/drive
Search string: hayward, ca


## <font color="blue">Data</font>

### Get API Keys

In [3]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + 'api_keys.csv')

# get keys
rapid_api_key = df_api_keys.loc[df_api_keys['API'] =='rapid']['KEY'].iloc[0] # replace this with your own key

In [4]:
rapid_api_key

'8b63d59cecmsha5a796447533ec2p11f828jsnf7fd2c8395a5'

In [5]:
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

In [6]:
headers = {
    'x-rapidapi-host': "zillow-com1.p.rapidapi.com",
    'x-rapidapi-key': rapid_api_key
    }
import requests
req = requests.request('GET', url, headers=headers)

## <font color="green">Zillow</font>

### <font color="purple">Endpoint: Property</font>

In [7]:
# get data
url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

querystring1 = {
               "location": "94611",
               "home_type":"Houses",
               "minPrice": "800000",
               "maxPrice": "1250000",
               "sqftMin": "1250",
               "bedsMin": "3"
               }

querystring2 = {
               "location": "94605",
               "home_type":"Houses",
               "minPrice": "800000",
               "maxPrice": "1250000",
               "sqftMin": "1250",
               "bedsMin": "3"
               }

querystring3 = {
               "location": "94619",
               "home_type":"Houses",
               "minPrice": "800000",
               "maxPrice": "1250000",
               "sqftMin": "1250",
               "bedsMin": "3"
               }

headers = {
    'x-rapidapi-host': "zillow-com1.p.rapidapi.com",
    'x-rapidapi-key': rapid_api_key
    }

z_for_sale_resp1 = requests.request("GET", url, headers=headers, params=querystring1)
z_for_sale_resp2 = requests.request("GET", url, headers=headers, params=querystring2)
z_for_sale_resp3 = requests.request("GET", url, headers=headers, params=querystring3)

# transform to json
z_for_sale_resp_json1 = z_for_sale_resp1.json()
z_for_sale_resp_json2 = z_for_sale_resp2.json()
z_for_sale_resp_json3 = z_for_sale_resp3.json()

In [10]:
len(z_for_sale_resp_json3.get('props', []))

19

In [33]:
# view data
df_z_for_sale1 = pd.json_normalize(data=z_for_sale_resp_json1['props'])
df_z_for_sale2 = pd.json_normalize(data=z_for_sale_resp_json2['props'])
df_z_for_sale3 = pd.json_normalize(data=z_for_sale_resp_json3['props'])

df_z_for_sale = pd.concat([df_z_for_sale1,df_z_for_sale2,df_z_for_sale3], ignore_index=True)

print('Num of rows:', len(df_z_for_sale))
print('Num of cols:', len(df_z_for_sale.columns))
#df_z_for_sale.head(50)

Num of rows: 60
Num of cols: 29


In [12]:
# download file (if needed)
#df_z_for_sale.to_csv('df_z_for_sale.csv')
#files.download('df_z_for_sale.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
# get zpids to a list
zpid_list = df_z_for_sale['zpid'].tolist()
zpid_list_new=zpid_list[:5]
len(zpid_list_new)

5

In [ ]:
# get property detail

# create empty list
prop_detail_list = []

# iterate through list of properties
for zpid in zpid_list_new:

  # end point
  url = "https://zillow-com1.p.rapidapi.com/property"

  querystring = {"zpid":zpid}

  # header
  headers = {
      'x-rapidapi-host': "zillow-com1.p.rapidapi.com",
      'x-rapidapi-key': rapid_api_key
      }

  # get property detail
  z_prop_detail_resp = requests.request("GET", url, headers=headers, params=querystring)
  #print(z_prop_detail_resp)
  z_prop_detail_resp_json = z_prop_detail_resp.json()

  # wait 1 sec based on limit
  #time.sleep(1.5)

  prop_detail_list.append(z_prop_detail_resp_json)

In [32]:
prop_detail_list

[{'message': 'You have exceeded the rate limit per minute for your plan, BASIC, by the API provider'},
 {'message': 'You have exceeded the rate limit per minute for your plan, BASIC, by the API provider'},
 {'message': 'You have exceeded the rate limit per minute for your plan, BASIC, by the API provider'},
 {'message': 'You have exceeded the rate limit per minute for your plan, BASIC, by the API provider'},
 {'message': 'You have exceeded the rate limit per minute for your plan, BASIC, by the API provider'}]

In [ ]:
# download file (if needed)
#df_z_for_sale.to_csv('df_z_for_sale.csv')
#files.download('df_z_for_sale.csv')

#df_z_prop_detail_output.to_csv('df_z_prop_detail_output.csv')
#files.download('df_z_prop_detail_output.csv')

In [24]:
# convert to dataframe
df_z_prop_detail = pd.json_normalize(prop_detail_list)
print('Num of rows:', len(df_z_prop_detail))
print('Num of cols:', len(df_z_prop_detail.columns))

Num of rows: 60
Num of cols: 313


In [19]:
# columns of interest
detail_cols = ['streetAddress',
 'city',
 'county',
 'zipcode',
 'state',
 'price',
 'homeType',
 'timeOnZillow',
 'zestimate',
 'rentZestimate',
 'livingArea',
 'bedrooms',
 'bathrooms',
 'yearBuilt',
 'description',
 'priceHistory',
 'taxHistory',
 'zpid'
 ]

# retain limited columns for output
df_z_prop_detail_output = df_z_prop_detail[detail_cols]
#df_z_prop_detail_output.dropna(subset=['streetAddress'],inplace=True)

In [37]:
df_z_for_sale['price']

,price
0,1195000
1,1200000
2,1098000
3,1095000
4,899000
5,1195000
6,999000
7,999888
8,979000
9,959900


## Zillow ZPID
Useful to retrieve zillow detailed data for ANY address

In [47]:
!rm example.txt
df_z_prop_detail_output['streetAddress'].iloc[0]

'5951 Grizzly Peak Blvd'

In [48]:
data = """Name,Address
"""

In [49]:
for x in range (0,len(df_z_prop_detail_output),1):
  # Step 1: Open the file in write mode
  file = open('example.txt', 'a')

  # Step 2: Write text to the file
  if x==0:
    file.write("Name,Address\n")
    file.write("House_"+str(x)+", "+df_z_prop_detail_output['streetAddress'].iloc[x]+"; "+df_z_prop_detail_output['city'].iloc[x]+" CA"+"\n")
  else:
    file.write("House_"+str(x)+", "+df_z_prop_detail_output['streetAddress'].iloc[x]+"; "+df_z_prop_detail_output['city'].iloc[x]+" CA"+"\n")
  # Step 3: Close the file
file.close()

In [50]:
#!rm ./example.txt
import pandas as pd
data=pd.read_csv('./example.txt')

In [51]:
df=data

In [38]:
import pandas as pd
import requests
import json
import urllib
import io

In [39]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="example app")

In [40]:
df=df_z_for_sale

In [41]:
df["loc"] = df['address'].apply(geolocator.geocode)

In [61]:
df

,dateSold,propertyType,lotAreaValue,address,variableData,priceChange,zestimate,imgSrc,price,detailUrl,bedrooms,contingentListingType,longitude,latitude,listingStatus,zpid,rentZestimate,daysOnZillow,bathrooms,livingArea,country,currency,lotAreaUnit,hasImage,listingSubType.is_FSBA,listingSubType.is_openHouse,listingSubType.is_foreclosure,listingSubType.is_bankOwned,listingSubType.is_forAuction,loc,point,lat,lon,altitude,price2
0,None,SINGLE_FAMILY,0.520000,"5951 Grizzly Peak Blvd, Oakland, CA 94611",None,NaN,NaN,https://photos.zillowstatic.com/fp/4366b950eae...,1195000,/homedetails/5951-Grizzly-Peak-Blvd-Oakland-CA...,3,None,-122.211570,37.853046,FOR_SALE,24819315,5972.0,11,3,2272,USA,USD,acres,True,True,NaN,NaN,NaN,NaN,"(5951, Grizzly Peak Boulevard, Oakland, Alamed...","(37.85304255, -122.21158604551943, 0.0)",37.853043,-122.211586,0.0,1.2M
1,None,SINGLE_FAMILY,6534.000000,"207 Taurus Ave, Oakland, CA 94611",None,NaN,1380000.0,https://photos.zillowstatic.com/fp/1e97f7ce0e7...,1200000,/homedetails/207-Taurus-Ave-Oakland-CA-94611/2...,4,None,-122.217514,37.839530,FOR_SALE,24818437,5028.0,4,3,1766,USA,USD,sqft,True,True,True,NaN,NaN,NaN,"(207, Taurus Avenue, Oakland, Alameda County, ...","(37.8395562, -122.21749807415506, 0.0)",37.839556,-122.217498,0.0,1.2M
2,None,SINGLE_FAMILY,6969.600000,"5630 Balboa Dr, Oakland, CA 94611",None,NaN,NaN,https://photos.zillowstatic.com/fp/37f22e49755...,1098000,/homedetails/5630-Balboa-Dr-Oakland-CA-94611/2...,3,None,-122.205986,37.831203,FOR_SALE,24816430,5684.0,66,2,1914,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,"(5630, Balboa Drive, Montclair, Oakland, Alame...","(37.83121455, -122.20598315537953, 0.0)",37.831215,-122.205983,0.0,1.1M
3,None,SINGLE_FAMILY,8276.400000,"1738 Indian Way, Oakland, CA 94611",None,NaN,NaN,https://photos.zillowstatic.com/fp/5eaa3aa1231...,1095000,/homedetails/1738-Indian-Way-Oakland-CA-94611/...,3,None,-122.207370,37.840214,FOR_SALE,24817631,NaN,3,2,1584,USA,USD,sqft,True,True,True,NaN,NaN,NaN,"(1738, Indian Way, Oakland, Alameda County, Ca...","(37.8402173, -122.2073642704905, 0.0)",37.840217,-122.207364,0.0,1.1M
4,None,SINGLE_FAMILY,8276.400000,"6206 Valley View Rd, Oakland, CA 94611",None,NaN,1097600.0,https://photos.zillowstatic.com/fp/fe2bb055c84...,899000,/homedetails/6206-Valley-View-Rd-Oakland-CA-94...,3,None,-122.210780,37.841976,FOR_SALE,24819051,5295.0,26,2,1502,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,"(6206, Valley View Road, Oakland, Alameda Coun...","(37.84195695, -122.21083382112457, 0.0)",37.841957,-122.210834,0.0,0.9M
5,None,SINGLE_FAMILY,0.250000,"684 Fairmount Ave, Oakland, CA 94611",None,NaN,NaN,https://photos.zillowstatic.com/fp/f826b7ea847...,1195000,/homedetails/684-Fairmount-Ave-Oakland-CA-9461...,3,None,-122.251816,37.822758,FOR_SALE,24747886,3125.0,3,2,1286,USA,USD,acres,True,True,NaN,NaN,NaN,NaN,"(684, Fairmount Avenue, Piedmont Avenue, Oakla...","(37.8227597, -122.25182527158438, 0.0)",37.822760,-122.251825,0.0,1.2M
6,None,SINGLE_FAMILY,6098.400000,"134 Moss Ave, Oakland, CA 94611",None,NaN,NaN,https://photos.zillowstatic.com/fp/5f6b880eb5f...,999000,/homedetails/134-Moss-Ave-Oakland-CA-94611/247...,3,None,-122.253760,37.821323,FOR_SALE,24744127,5099.0,13,2,2239,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,"(134, Moss Avenue, Harrison/Oakland, Oakland, ...","(37.82131785, -122.25378656433486, 0.0)",37.821318,-122.253787,0.0,1.0M
7,None,SINGLE_FAMILY,6969.600000,"6684 Banning Dr, Oakland, CA 94611",None,NaN,NaN,https://photos.zillowstatic.com/fp/23e3c62123d...,999888,/homedetails/6684-Banning-Dr-Oakland-CA-94611/...,5,None,-122.191560,37.831688,FOR_SALE,24815310,5682.0,25,3,2452,USA,USD,sqft,True,True,NaN,NaN,NaN,NaN,"(6684, Banning Drive, Oakland, Alameda County,...","(37.8316814, -122.19155820104427, 0.0)",37.831681,-122.191558,0.0,1.0M
8,None,SINGLE_FAMILY,3920.400000,"4250 Terrace St, Oakland, CA 94611",None,NaN,NaN,https://photos.zillowstatic.com/fp/67b42f3ee3f...,979000,/homedetails/4250-Terrace-St-Oakland-CA-94611/...,4,None,-122.253020,37.830

In [43]:
df.dropna(subset=['loc'],inplace=True)

In [56]:
df["price2"] = df['price'].apply(lambda x: f'{x/10**6:.1f}M')

In [67]:
df["address_price"] = df['address'] + ', ' + df['price2']

In [58]:
df["point"]= df["loc"].apply(lambda loc: tuple(loc.point) if loc else None)

In [59]:
df[['lat', 'lon', 'altitude']] = pd.DataFrame(df['point'].to_list(), index=df.index)

In [68]:
# import the library and its Marker clusterization service
import folium
from folium.plugins import MarkerCluster
# Create a map object and center it to the avarage coordinates to m
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=2)
# if the points are too close to each other, cluster them, create a cluster overlay with MarkerCluster, add to m
marker_cluster = MarkerCluster().add_to(m)
# draw the markers and assign popup and hover texts
# add the markers the the cluster layers so that they are automatically clustered
for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    folium.Marker(location=location,

                      tooltip=r['address_price'])\
    .add_to(marker_cluster)
# display the map
m

In [61]:
df

,Name,Address,loc,point,lat,lon,altitude
0,House_0,5951 Grizzly Peak Blvd; Oakland CA,"(5951, Grizzly Peak Boulevard, Oakland, Alamed...","(37.85304255, -122.21158604551943, 0.0)",37.853043,-122.211586,0.0
1,House_1,207 Taurus Ave; Oakland CA,"(207, Taurus Avenue, Oakland, Alameda County, ...","(37.8395562, -122.21749807415506, 0.0)",37.839556,-122.217498,0.0
2,House_2,5630 Balboa Dr; Oakland CA,"(5630, Balboa Drive, Montclair, Oakland, Alame...","(37.83121455, -122.20598315537953, 0.0)",37.831215,-122.205983,0.0
3,House_3,1738 Indian Way; Oakland CA,"(1738, Indian Way, Oakland, Alameda County, Ca...","(37.8402173, -122.2073642704905, 0.0)",37.840217,-122.207364,0.0
4,House_4,6206 Valley View Rd; Oakland CA,"(6206, Valley View Road, Oakland, Alameda Coun...","(37.84195695, -122.21083382112457, 0.0)",37.841957,-122.210834,0.0
5,House_5,684 Fairmount Ave; Oakland CA,"(684, Fairmount Avenue, Piedmont Avenue, Oakla...","(37.8227597, -122.25182527158438, 0.0)",37.822760,-122.251825,0.0
6,House_6,134 Moss Ave; Oakland CA,"(134, Moss Avenue, Harrison/Oakland, Oakland, ...","(37.82131785, -122.25378656433486, 0.0)",37.821318,-122.253787,0.0
7,House_7,6684 Banning Dr; Oakland CA,"(6684, Banning Drive, Oakland, Alameda County,...","(37.8316814, -122.19155820104427, 0.0)",37.831681,-122.191558,0.0
8,House_8,4250 Terrace St; Oakland CA,"(4250, Terrace Street, Broadway Auto Row, Nort...","(37.830924350000004, -122.25305057518327, 0.0)",37.830924,-122.253051,0.0
9,House_9,8040 Shepherd Canyon Rd; Oakland CA,"(8040, Shepherd Canyon Road, Oakland, Alameda ...","(37.83576495, -122.18586692939164, 0.0)",37.835765,-122.185867,0.0


In [ ]:
#https://towardsdatascience.com/pythons-geocoding-convert-a-list-of-addresses-into-a-map-f522ef513fd6#5352

# End Notebook